In [ ]:
import pandas as pd
import os
import glob


In [ ]:
def get_subjects():
    global subjects
    global basedir
    global sess_ID
    subjects = []
    # make script argument?
    #basedir=input("Enter the main directory: ")
    basedir='/projects/niblab/bids_projects/Experiments/ChocoData'
    ## add this as input variable
    multi_sess= True
    
    sess_ID = "ses-4"
    if multi_sess == True:
        fmriprep_dir=os.path.join(basedir, 'fmriprep')
    else:
        fmriprep_dir=os.path.join(basedir, 'fmriprep')
    subjects=glob.glob(os.path.join(fmriprep_dir, "sub-*"))
    #print("SUBJECTS: ", subjects)

In [ ]:
def write_files(task, moco_df, outputdir, sub):
    # iterate through the motion correction data frame by columns,
    # writing individual columns to individual files
    for col in moco_df.columns:
        filename = "%s_%s_%s_%s.txt"%(sub, sess_ID, task, col)
        print("WRITING TO FILE >>>>>>>>>>>>>>>>>>>>>>> %s"%filename)
        output_path=os.path.join(outputdir, filename)
        print("Writing to file, ", output_path)
        moco_df[col].to_csv(output_path, header=False, index=False)

In [ ]:

def get_data():
    errors = []
    for sub_path in subjects:
        try:
            sub = sub_path.split("/")[-1]
            print("--------------> GETTING MOCOS FOR SUBJECT: ", sub)
            filepath=os.path.join(sub_path, sess_ID,'fmriprep', sub, sess_ID,'func')
            outputdir=os.path.join(basedir, 'derivatives', sub, sess_ID, 'func', 'motion_assessment')
            outputdir=os.path.join(outputdir, 'motion_parameters')
            print(">>>>>>>FILEPATH: %s >>>>>>>>OUTPUT DIRECTORY: %s"%(filepath, outputdir))
            os.chdir(filepath)
            for file in glob.glob("*confounds.tsv"):
                print("---------------------------------------------> GRABBING NEW FILE:")
                print("FILE: ", file)
                df = pd.read_table(file)
                moco_df=df[['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]
                moco_df.columns = ['moco0', 'moco1', 'moco2', 'moco3', 'moco4', 'moco5']
                print("DATAFRAME: \n ", moco_df.head())
                name=file.split('_')
                #print("NAME: ", name)
                for word in name:
                    if 'task' in word:
                        task=word
                        print("TASK: ", task)
                        write_files(task, moco_df, outputdir,sub)
        except FileNotFoundError as not_found:
            print("********************FILE NOT FOUND: ", not_found.filename)
            if sub not in errors:
                errors.append(sub)
        #print("ERRORS ", errors)
        #print("ERRORS SORTED ", sorted(errors))
        errors = sorted(errors)
    for err in errors:
            #print("ERROR" + err)
        file = basedir+"/error_files_ses-1.txt"
        with open(file, 'a') as f:
            f.write("--------------------------------> FILE NOT FOUND FOR SUBJECT: " + err  + "\n")
            f.close()

In [ ]:
def main():
    get_subjects()
    get_data()
main()